In [1]:
import cv2
import numpy as np
import dlib
from math import hypot

In [3]:
from tensorflow.keras.models import load_model

In [2]:
import pickle
knn = pickle.load(open('knn.pickle','rb'))
svc_model = pickle.load(open('svc_model.pickle','rb'))

In [4]:
CNN_model = load_model('CNN_model.h5')

print(CNN_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 154, 154, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 77, 77, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 165888)            0

In [6]:
Vgg_model = load_model('Vgg_model.h5')

print(Vgg_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0     

In [5]:
import pickle

X_train=pickle.load(open('X_train.pickle','rb'))
y_train=pickle.load(open('y_train.pickle','rb'))

X_test=pickle.load(open('X_test.pickle','rb'))
y_test=pickle.load(open('y_test.pickle','rb'))

In [7]:
IMG_SIZE=160

In [69]:
def VGG_frame_shape(img):
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img_new = img.reshape((-1,img.shape[0],img.shape[1],3))
    return img_new


In [70]:
def CNN_frame_shape(img):
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img_new = img.reshape((-1,img.shape[0],img.shape[1],1))
    return img_new

In [71]:
def func_CNN(pred):
    chk=pred[0][0]
    if chk < 0.01:
        result = "fake"
    else:
        result = "real"
    return result

In [72]:
def func(pred):
    if pred==0:
        return "fake"
    else :
        return "real"
    

In [73]:
def frame_shape(img):
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img_new = img.reshape((1,img.shape[0]*img.shape[1]))
    return img_new


In [74]:
images = list()
def Convert_Image(my_img):
    
    my_img = cv2.cvtColor(my_img,cv2.COLOR_BGR2GRAY)
    
    images.append(my_img)
    
    vgg_img = cv2.cvtColor(my_img,cv2.COLOR_BGR2RGB)
    
    images.append(vgg_img)
    
    return images
    

In [75]:
def Process_Roi(my_img):
    roi = cv2.resize(my_img, (160, 160))
    roi=roi/255
    
    return roi

In [76]:
def KNN_MODEL(roi):
    
    f_img=frame_shape(roi)
    
    pred = knn.predict(f_img)
    
    var=func(pred)

    return var

In [77]:
def SVM_MODEL(roi):
    
    f_img=frame_shape(roi)
    
    svc_pred = svc_model.predict(f_img)
    
    var2=func(svc_pred)
    
    return var2
    

In [84]:
def CNN_MODEL(roi):
    
    f_img=CNN_frame_shape(roi)
    
    pred = CNN_model.predict(f_img)

    var3=func(pred)
    
    return var3

In [85]:
def VGG_MODEL(roi):
    f_img=VGG_frame_shape(roi)
    
    pred = Vgg_model.predict(f_img)

    var4=func_CNN(pred)
    
    return var4

In [86]:
predictions=dict()
def Prediction_Models(my_img,vgg_img):
    
    roi = Process_Roi(my_img)
    roi2 = Process_Roi(vgg_img)
    
    var = KNN_MODEL(roi)
    var2 = SVM_MODEL(roi)
    var3 = CNN_MODEL(roi)
    var4 = VGG_MODEL(roi2)
    
    
    predictions['KNN'] = var
    predictions['SVM'] = var2
    predictions['CNN'] = var3
    predictions['VGG'] = var4
    
    return predictions

In [87]:
def Integrated_Output(img):
    images_list = Convert_Image(img)
    
    my_img = images_list[0]
    vgg_img = images_list[1]
    
    final_results = Prediction_Models(my_img,vgg_img)
    
    return final_results

In [88]:
#my_img=cv2.imread("E:\Python\Spoofing-Version02\Fake-Hannan.jpeg")
my_img=cv2.imread("E:\Python\liveness_detection\justin_fake.jpg",cv2.IMREAD_GRAYSCALE)

In [89]:
results_after_prediction = Integrated_Output(my_img)
print(results_after_prediction)

{'KNN': 'fake', 'SVM': 'fake', 'CNN': 'fake', 'VGG': 'real'}


In [90]:
for values in results_after_prediction.values():
    #print(values)
    

fake
fake
fake
real
